In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "eryk-mazus/polka-1.1b"
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.generation_config.pad_token_id = tokenizer.pad_token_id
model.eval()

system_prompt = """
Wstęp:
Użytkownik nawiązuje kontakt z asystentem. Asystent jest pomocny i odpowiada zwięźle.

Dialog:"""


print ()
while (user_prompt := input().strip()):

    prompt = f"""
{system_prompt}
Użytkownik: \"{user_prompt}\"
Asystent: \""""


    model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
    generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=126,
      do_sample=True,
      penalty_alpha=0.6,
      top_k=10
    )

    output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print (50 * '-', flush=True)
    print (f"Użytkownik: \"{user_prompt}\"", flush=True)

    response = output.removeprefix(prompt).strip().replace("\\n", "\n").split("\n")[0].split("\"")[0] + "\""
    system_prompt = prompt + response
    print (f"Chatbot: \"{response}", flush=True)


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


--------------------------------------------------
Użytkownik: "Cześć!"
Chatbot: "Cześć."
--------------------------------------------------
Użytkownik: "Jaki jest twój ulubiony kolor?"
Chatbot: "Pomarańczowy."
--------------------------------------------------
Użytkownik: "Mój też!"
Chatbot: "Nie ma sprawy."
